# Functional Programming in Python
## Introduction
Functional programming is an approach to programming where programs are built by composing and running functions that perform a series of transformation on data. This is in contrast with the common approach of *imperative programming*, where programs are written as series of statements which modify the *state* of the computation environment. Normally within functional programming, great focus is placed on *composability*, *immutability* and  *purity*. We are going to define these terms in more detail later.
## Why Functional Programming
Why do we choose functional programming? There are a series of advantages to this approach, namely:
- **Debugging** and **testing** are easy: there are no surprises because every function only does one thing and does not affect any other piece of the program.
- **Parallelisation** is trivial: because functions are just small boxes that take one input and produce an output and do not depend implicitely on other parts of the code, it is easy to make several functions run in parallel.
## The basic principles of functional programming
All modern programming languages have *functions* (or methods, procedures, subroutines, subprograms); these are groups of program statements that perform a certain computation. Functions are defined once for the whole program and can be reused at will throughout the program whenever we need to perform the specific computation they are defined for. Using functions, we can split our code in smaller units that are only responsible for a specific *functionality*; this helps us structuring our code in a clean and understandable form. 
### Pure Functions
In functional programming, we try to strive for *purity*, that is we want to define and use functions that only depend on their input, always return the same output for the same inputs and do not have any *side effects*, that is they do not indirectly affect any other part of our program. You can think of these functions as mathematical functions. 
Other examples of side effects are:
- printing to the program output
- reading or writing files
- generating and using random numbers
To besser understand this concept, let us look at the function `my_first_pure_function` we defined below:


In [1]:
def my_first_pure_function(x: int) -> int: 
    return x + 1

&#x1F600
Is this a pure function?
As you could probably imagine, this function is pure. Any time we run it, we get the same  result and running the function does not affect any other part of our program:

In [2]:
x = 1
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")

x is 1
The result of calling the function is 2
x is 1
The result of calling the function is 2
x is 1


As a rule of thumb, any function that does not modify variables outside of its *scope* and only uses mathematical functions is a pure function.
Now, consider this function instead:

In [3]:
x = ["short", "list"] 
def do_something(y: str) -> None:
    x.append(y)

> &#x1F600 Is this function pure? 

Let's try and run it. 

In [4]:
print(x)
do_something("a")
print(x)

['short', 'list']
['short', 'list', 'a']


As you see from the output, the function modified the list `x`. Therefore, this function is not pure. This leads us to the next principle, **immutability**.

### Immutability
When writing programs in functional style, we usually avoid functions like `do_something`. Instead of modifying existing data (*mutation*), you write functions that transform your data and return new objects.
In the case of the function above, we would rewrite it as follows:


In [5]:
x = ["short", "list"] 
def do_something_immutable(x: list[str], y: str) -> list[str]:
    return x + [y]

print(x)
print(do_something_immutable(x, "a"))
print(x)

['short', 'list']
['short', 'list', 'a']
['short', 'list']


The output shows that `do_something_immutable` does not change `x`. It also does not reference to `x` outside of the scope but takes it as an argument. Instead of modfying the original `x` list, it returns a new list. When we want to keep immutability, this is the style we work with.

### Higher Order Functions / Functions as Values
Another important principle of functional programming is that **functions are values**. In programming languages (like python) that support a functional style, we can manipulate functions with the language, we can pass them around in a variable and even use functions as parameters for another function.

As an example, consider the function `function_caller`:



In [11]:
from typing import Callable, Any
def function_caller(f: Callable[[Any], Any], arg: Any) -> Any:
    print(f"Calling the function {f} with argument {arg}")
    res = f(arg)
    print(f"The result of the call was {res}")
    return res

This function takes a function `f` and its argument `args` as input and return the result of calling the function, while additionally printing a message on the standard output of the program. Let's try this out.
To do so, we define a new function `add_five`:

In [7]:
def add_five(x: int) -> int:
    return x + 5

Now we can call `function_caller` with `add_five` and another number as arguments:

In [13]:
function_caller(add_five, 4)

Calling the function <function add_five at 0x7fb68848f1c0> with argument 4
The result of the call was 9


9

This example was a bit convoluted, but it shows that in python we can use functions as values and even pass them as arguments to other functions.

This is actually useful in many cases; a typical example being numerical optimisation, where we want to find the parameters to a function that minimise a certain criteria.

Other than these specific use cases, there are some common *higher order functions*  (HoF), or functions that take functions as parameters that are common in most programming languages. 





## Mapping / Iteration 

### Using map and filter

Python offers the `map` function which can be used to apply a given function to all members of an `iterable`. Do not worry too much about what an `iterable` exactly is, just see it as a generalisation of a list, representing all objects that we can `iterate` over, accessing their values one by one. Let's see an example of `map` using the `add_five` function:

In [14]:
map(add_five, [1,2,3,4])

The output is a bit confusing! The reason is that `map` does not return a list but a [map](https://docs.python.org/3/library/functions.html#map) object. This object is `lazy`, that means that the values are only generated when we access them, for example by iterating over the `map` object. 
Fortunately, we can easily convert this into a list by calling the `list` function:

In [15]:
list(map(add_five, [1,2,3,4]))

[6, 7, 8, 9]

This is equivalen to using a for loop:


In [16]:
a = []
for i in [1,2,3,4]:
    a.append(add_five(i))
print(a)

[6, 7, 8, 9]


However, as you can see this former style is not very functional because we mutate an existing list `a` by appending the results of calling `add_five` to it. We will see later that python offers a syntax called *list compehrension* to express this operation in a more functional style.
`map` is very useful when we want to apply the same functions to a list of parameters and we do not want to write a `for` loop. It makes for more coincise code, while sacrificing some readability. 

#### Filtering
Another basic higher order function is `filter`. As the name says, this function is used to filter an `iterable` using a *predicate function*. This is a function that takes a value and return `true` or `false`. With `true`, the current element is kept, with `false` it is discarded.
We now try to write a predicate function that only keeps even numbers.

<a name="filter-example"></a>

In [18]:
def is_even(x: int) -> bool:
    return x % 2 == 0

Now we are ready to try `filter` using `is_even`. Because `filter` returns a `filter` object, we wrap it in `list` to directly see the result as a list:

In [20]:
list(filter(is_even, [1,2,3,4,5]))

[2, 4]

#### Reducing
A third basic HoF is *reduction*. This is a function that takes a function `f(x, y)` of two arguments and an iterable `l` an applies the function  to every element in the iterable *cumulatively* to produce one value. It works in the following way:
- The first argument of `f`, `x` is the current value of the accumulation. At first, this corresponds to the first element of `l`
- The second argument of `f`. `y` is the current element of the iterable. At first, this corresponds to the second element of `l`

Because of this behavior, this function is useful to compute sums or similar aggregation over a list of numbers. In python, this function is available in the `functools` [package](https://docs.python.org/3/library/functools.html) as a part of the python standard library.

As an example of using `reduce`, consider the following snippet:


In [4]:
import functools


def spy(x: int, y: int) -> int:
    print(f"x: {x}, y: {y}")
    return x * y

result = functools.reduce(spy, [1, 2, 3, 4, 5])
print(result)

x: 1, y: 2
x: 2, y: 3
x: 6, y: 4
x: 24, y: 5
120


As you can see, we are computing the cumulative product of `[1, 2, 3, 4, 5]`, that is the *factorial* of 5, or `5!`

### List Compehrensions
Many of the operations in the previous section can be performed in a different (some would say more *pythonic*) way using [list compehrensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).
These look like mini for-loops inside square brackets and are very useful to write programs in a functional style while keeping code more readable than using higher order functions.  

For example, if we want to double all integers between 1 and 10 and store the result in a list,  we can proceed in the classical imperative way:

In [13]:
doubles = []

for i in range(10):
    doubles.append(i * 2)

print(f"The doubles are {doubles}")


The doubles are [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Or we can use `map` like this:

In [19]:
doubles = map(lambda x: x * 2, range(10))

print(list(doubles))


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Again here we wrapped `doubles` in `list` because `map` is lazy and does not return a list immediately but a `map` object which can be converted into a list using `list`. 
This solution is more elegant and avoids side effects because we do not modify the previously defined `doubles` list, instead we directly produce a new list with the desired result.

Now, we solve the same problem using a list compherension:

In [20]:
doubles = [i * 2 for i in range(10)]
print(doubles)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


This looks a lot like the solution above for the for loop, but it is an *expression* instead of a *statement*. [Expressions](https://www.baeldung.com/cs/expression-vs-statement) in programming are pieces of code that return a value and hence can be composed together to form more complex expressions and used anywhere a value is needed. In functional style, we prefer working with expressions instead of statements because we try to keep [*referential transparency*](https://en.wikipedia.org/wiki/Referential_transparency).

Compherensions can do more than just iterate over a list. We can use them to filter elements of a list by adding an `if` statement at the end.
For example, consider the example of filtering even numbers  from a list [above](#filter-example); we can rewrite this with list compherensions like this:


In [21]:
def is_even(x: int) -> bool:
    return x % 2 == 0

[i for i in range(5) if is_even(i)]

[0, 2, 4]



## Exercises


### Pure or impure functions? 🌶️

For each of the functions below, determine whether they are pure or impure

In [6]:
def fun1(a: "list[str]") -> None:
    a.append("b")


def fun2(a: int) -> int:
    return a + 2


def fun3(a: "dict[str, str]"):
    a["test"] = "dest"



### Keeping only multiples of n 🌶️

Given a list `L` of integers, write a function that only keeps the numbers that are multiples of a given constant `k`.

- Example 1: given `nums = [1, 2, 3, 4, 5]`, and `k = 2`, the result must be `[2, 4]`

- Example 2: given `nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] ` and `k = 5`, the result is `[5, 10]`

### Flattening list of lists 🌶️🌶️

Imagine we receive a list of lists `L` like `[[1, 2], [3, 4]]`. Write a function `flatten` that converts this list into a `flat` list like `[1, 2, 3, 4]`. 


In [10]:
print(functools.reduce(lambda x,y: x + y, [[1,2,3], [1,2]]))

[1, 2, 3, 1, 2]
